In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [3]:
news = pd.read_csv("data/tesla_news.csv", index_col='date', parse_dates=['date'])
news = news.sort_index()

C:\Users\Olaf\AppData\Roaming\Python\Python36\site-packages\pandas\core\sorting.py:257: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  items = np.asanyarray(items)


In [13]:
time= "H"

In [14]:
prices = pd.read_csv("data/tesla_5m_20190201_20190522.csv", index_col='datetime', parse_dates=['datetime'])
prices.index = prices.index.tz_localize("US/Eastern").tz_convert("UTC")

In [15]:
prices = pd.concat([prices['open'].resample(time).agg("first"),
    prices['close'].resample(time).last(),
    prices['high'].resample(time).max(),
    prices['low'].resample(time).min(),
    prices['marketVolume'].resample(time).sum()], axis=1)
prices = prices.dropna()

In [24]:
new_index = []
for i, time in enumerate(news.index.round("H")):
    if time < news.index[i]:
        new_time = time + dt.timedelta(hours=1)
    else:
        new_time = time
    if new_time not in prices.index:
        new_index.append(prices.index[prices.index > new_time].min())
    else:
        new_index.append(new_time)
news.index = new_index


In [25]:
news = news.loc[pd.notnull(news.index)]

In [16]:
tweets = pd.read_csv('data/tesla_cleaned.csv', header=None, 
        names=['id','text','created_at','favorite_count','retweet_count','user','screen_name', 
               'cleaned_text'], parse_dates=['created_at'], index_col=['created_at'])

tweets = tweets.sort_index()

In [18]:
tweets.index = tweets.index.tz_localize("UTC")

In [19]:
new_index = []
for i, time in enumerate(tweets.index.round("H")):
    if time < tweets.index[i]:
        new_time = time + dt.timedelta(hours=1)
    else:
        new_time = time
    if new_time not in prices.index:
        new_index.append(prices.index[prices.index > new_time].min())
    else:
        new_index.append(new_time)
tweets.index = new_index


In [20]:
tweets = tweets.loc[pd.notnull(tweets.index)]

In [22]:
prices_tweets = prices.join([tweets])
prices_news = prices.join([news])

In [24]:
prices_tweets.to_csv("data/tweets_prices.csv")
prices_news.to_csv("data/news_prices.csv")